# Keras Overview

An overview of using Keras 3 for building an autoencoder and using it for anomaly detection.

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [3]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [4]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [5]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('jax','jax'),
    ('keras', 'keras', '3.6.0'),
    ('tensorflow', 'tensorflow'),
    ('pydot', 'pydot')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Graphviz Install

Plotting the [model structure with Keras](https://keras.io/api/utils/model_plotting_utils/) uses [Graphviz](https://graphviz.org/download/).

This code check for Graphviz and if missing installs it.

In [6]:
check = !dot -V
if check[0].startswith('dot'):
    print(f'Graphviz installed with version: {check[0]}')
else:
    print('Installing Graphviz...')
    install = !sudo apt-get install graphviz --assume-yes
    print('Completed')

Graphviz installed with version: dot - graphviz version 2.43.0 (0)


### API Enablement

In [7]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [8]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [9]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [10]:
REGION = 'us-central1'
SERIES = 'frameworks-keras'
EXPERIMENT = 'overview'

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

# make this the BigQuery Project / Dataset / Table prefix to store results
BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-', '_')
BQ_TABLE = EXPERIMENT
BQ_REGION = REGION[0:2] # use a multi region

Packages

In [11]:
import os#, json, time, glob

#import numpy as np

# import keras, set backend prior to first import
os.environ['KERAS_BACKEND'] = 'jax'
import keras
import tensorflow as tf

# Vertex AI
from google.cloud import aiplatform

# BigQuery
from google.cloud import bigquery

2024-12-12 21:05:35.075082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734037535.124003 1064930 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734037535.137007 1064930 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
aiplatform.__version__

'1.71.0'

In [13]:
keras.__version__

'3.6.0'

In [14]:
tf.__version__

'2.18.0'

Clients

In [15]:
# vertex ai clients
aiplatform.init(project = PROJECT_ID, location = REGION)

# bigquery client
bq = bigquery.Client(project = PROJECT_ID)

---
## Review Source Data

This is a BigQuery public table of 284,807 credit card transactions classified as fradulant or normal in the column `Class`.
- The data can be researched further at this [Kaggle link](https://www.kaggle.com/mlg-ulb/creditcardfraud).
- Read mode about BigQuery public datasets [here](https://cloud.google.com/bigquery/public-data)

In order protect confidentiality, the original features have been transformed using [principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) into 28 features named `V1, V2, ... V28` (float).  Two descriptive features are provided without transformation by PCA:
- `Time` (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- `Amount` (float) is the value of the transaction
 

### Review BigQuery table:

In [16]:
source_data = bq.query(f'SELECT * FROM `{BQ_SOURCE}` LIMIT 5').to_dataframe()
source_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8748.0,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.0,0
1,27074.0,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.0,0
2,28292.0,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.0,0
3,28488.0,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.0,0
4,31392.0,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.0,0


In [17]:
source_data.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       Int64
dtype: object

---
## Prepare Data Source

The data preparation includes adding splits for machine learning with a column named `splits` with 80% for training (`TRAIN`), 10% for validation (`VALIDATE`) and 10% for testing (`TEST`).  Additionally, a unique identifier was added to each transaction, `transaction_id`. 

### Create/Recall Dataset

In [18]:
dataset = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
dataset.location = BQ_REGION
bq_dataset = bq.create_dataset(dataset, exists_ok = True)

### Create/Recall Table With Preparation For ML

Copy the data from the source while adding columns:
- `transaction_id` as a unique identify for the row
    - Use the `GENERATE_UUID()` function
- `splits` column to randomly assign rows to 'TRAIN", "VALIDATE" and "TEST" groups
    - stratified sampling within the levels of `class` by first assigning row numbers within the levels of `class` then using the with a CASE statment to assign the `splits` level.

In [19]:
job = bq.query(f"""
#CREATE OR REPLACE TABLE
CREATE TABLE IF NOT EXISTS 
    `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` AS
WITH
    add_id AS (
        SELECT *,
            GENERATE_UUID() transaction_id,
            ROW_NUMBER() OVER (PARTITION BY class ORDER BY RAND()) as rn
            FROM `{BQ_SOURCE}`
    )
SELECT * EXCEPT(rn),
    CASE 
        WHEN rn <= 0.8 * COUNT(*) OVER (PARTITION BY class) THEN 'TRAIN'
        WHEN rn <= 0.9 * COUNT(*) OVER (PARTITION BY class) THEN 'VALIDATE'
        ELSE 'TEST'
    END AS splits
FROM add_id
""")
job.result()
(job.ended-job.started).total_seconds()

0.509

In [20]:
raw_sample = bq.query(f'SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` LIMIT 5').to_dataframe()
raw_sample

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,43330.0,-1.510308,0.780684,2.085747,3.123133,0.589557,1.301681,-1.300991,-2.345333,0.148722,...,-0.341324,-0.370869,-0.201199,0.423964,0.210998,-0.058616,0.0,0,34a1f550-443c-45f0-858d-94cf0baebfe8,TEST
1,141955.0,-1.585532,1.240411,-0.558349,-0.841657,0.397502,-1.532723,0.388818,0.575896,-0.278370,...,-0.249493,0.001462,-0.045791,-0.110667,0.305249,0.015271,0.0,0,a2dfc3ff-aa20-4742-bb1c-c6f67440e34a,TEST
2,58813.0,-2.524342,-0.313784,2.304085,3.043016,5.495686,-2.600347,-3.779679,-1.042154,-1.014503,...,-6.185491,0.440080,-1.409641,-0.209465,-0.111065,0.116788,0.0,0,d7910b3a-fc5e-417c-8b61-5516cc4dc981,TEST
3,139716.0,-2.006121,2.048709,-0.963971,-1.046216,-0.118273,-1.400030,0.401296,0.728337,0.095944,...,0.059168,-0.113542,0.066162,-0.143905,-0.218363,-0.091904,0.0,0,4de5ecd4-7c82-40dd-a52a-4b5e8b1e8ba3,TEST
4,59183.0,1.175167,-0.087900,0.278759,-0.279214,-0.166236,0.058772,-0.232681,0.192241,-0.014614,...,0.165047,-0.253526,-0.016523,0.890504,-0.059400,-0.013996,0.0,0,ec122bde-5780-4223-9e35-7f26de453512,TEST


### Review the number of records for each level of `Class` for each of the data splits:

In [21]:
bq.query(f"""
SELECT splits, class,
    count(*) as count,
    ROUND(count(*) * 100.0 / SUM(count(*)) OVER (PARTITION BY class), 2) AS percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
GROUP BY splits, class
""").to_dataframe()

,splits,class,count,percentage
0,TEST,1,50,10.16
1,TRAIN,1,393,79.88
2,VALIDATE,1,49,9.96
3,TEST,0,28432,10.00
4,TRAIN,0,227452,80.00
5,VALIDATE,0,28431,10.00


## Training - Local, Simple

---
Data, Readers, Prep Function, and Normalizers

In [22]:
train_ds = bq.query(f"SELECT * EXCEPT(splits) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'TRAIN'").to_dataframe()
test_ds = bq.query(f"SELECT * EXCEPT(splits) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'TEST'").to_dataframe()
validate_ds = bq.query(f"SELECT * EXCEPT(splits) FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE splits = 'VALIDATE'").to_dataframe()

In [23]:
train_ds.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class', 'transaction_id'],
      dtype='object')

In [24]:
# Define column categories
var_class = ['Class']
var_omit = ['transaction_id']
var_numeric = [x for x in train_ds.columns.tolist() if x not in var_class + var_omit]

In [25]:
def reader(ds):
    return tf.data.Dataset.from_tensor_slices(dict(ds))

In [26]:
train_read = reader(train_ds)
validate_read = reader(validate_ds)
test_read = reader(test_ds)

2024-12-12 21:06:18.678516: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [27]:
batch = next(iter(train_read.batch(10).take(1)))
batch.keys()

dict_keys(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class', 'transaction_id'])

In [28]:
batch['Time']

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([ 83533., 113822.,  39200.,  61781., 148437., 123042., 127907.,
         1888., 117742., 163935.])>

In [29]:
def prep_batch(source):
    for k in var_omit + var_class:
        source.pop(k, None)
    numeric_values = tf.stack([source[col] for col in var_numeric], axis=-1)
    return numeric_values, numeric_values

In [30]:
batch, batch = next(iter(train_read.map(prep_batch).batch(10).take(1)))

In [31]:
batch.shape

TensorShape([10, 30])

In [32]:
batch[0]

<tf.Tensor: shape=(30,), dtype=float64, numpy=
array([ 8.35330000e+04,  1.12552389e+00,  1.89853946e-01,  1.50865969e+00,
        2.71993690e+00, -8.82341741e-01,  1.28390258e-01, -6.22650925e-01,
        2.19444433e-01,  2.55370444e-01,  5.21744641e-01, -1.11755689e+00,
       -3.42581768e-01, -9.18035487e-01, -2.24182825e-01, -1.98108066e-01,
        6.65628330e-01, -3.60429717e-01, -8.31641571e-02, -8.35711759e-01,
       -2.25172417e-01, -9.75838741e-02, -1.72683342e-01,  5.75558670e-02,
        3.60859312e-01,  2.65443187e-01, -4.04174990e-02,  3.66861302e-02,
        3.36852774e-02,  0.00000000e+00])>

In [33]:
# Normalization
normalizer = keras.layers.Normalization(axis=-1, name='normalization')

# Adapt the normalizer 
numeric_feature_reader = train_read.prefetch(10).map(prep_batch).batch(1000)
normalizer.adapt(numeric_feature_reader.map(lambda x, _: x))

# Denormalization using normalizer parameters
denormalizer = keras.layers.Normalization(axis = -1, name = 'denormalize', invert = True, mean = normalizer.mean, variance = normalizer.variance)

---
Autoencoder #1 - Bones

In [34]:
autoencoder_input = keras.Input(shape = (len(var_numeric),), name = "autoencoder_input")
normalized_input = normalizer(autoencoder_input)
encoder = keras.layers.Dense(16, activation='relu', name='enc_dense1')(normalized_input)
encoder = keras.layers.Dropout(0.2, name='enc_dropout1')(encoder)
encoder = keras.layers.Dense(8, activation='relu', name='enc_dense2')(encoder)
encoder = keras.layers.Dropout(0.2, name='enc_dropout2')(encoder)
encoder = keras.layers.Dense(4, activation='relu', name='enc_dense3')(encoder)
decoder = keras.layers.Dense(8, activation='relu', name='dec_dense1')(encoder)
decoder = keras.layers.Dropout(0.2, name='dec_dropout1')(decoder)
decoder = keras.layers.Dense(16, activation='relu', name='dec_dense2')(decoder)
decoder = keras.layers.Dropout(0.2, name='dec_dropout2')(decoder)
decoder = keras.layers.Dense(30, activation='linear', name='dec_dense3')(decoder)
reconstructed = denormalizer(decoder)

In [35]:
autoencoder = keras.Model(autoencoder_input, reconstructed, name = 'autoencoder')

In [36]:
autoencoder.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.MeanAbsoluteError(),
    metrics = [
        keras.metrics.RootMeanSquaredError(name = 'rmse'),
        keras.metrics.MeanSquaredError(name = 'mse'),
        keras.metrics.MeanAbsoluteError(name = 'mae'),
        keras.metrics.MeanSquaredLogarithmicError(name = 'msle')
    ]
)

In [37]:
autoencoder.summary()

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ autoencoder_input (InputLayer)  │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization (Normalization)   │ (None, 30)             │            61 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense1 (Dense)              │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dropout1 (Dropout)          │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense2 (Dense)              │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dropout2 (Dropout)          │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense3 (Dense)              │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dense1 (Dense)              │ (None, 8)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dropout1 (Dropout)          │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dense2 (Dense)              │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dropout2 (Dropout)          │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dense3 (Dense)              │ (None, 30)             │           510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ denormalize (Normalization)     │ (None, 30)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,423 (5.56 KB)

 Trainable params: 1,362 (5.32 KB)

 Non-trainable params: 61 (244.00 B)

In [38]:
# Data preparation for training
train_dataset = train_read.prefetch(10).map(prep_batch).batch(100)
val_dataset = validate_read.prefetch(10).map(prep_batch).batch(100)

In [39]:
history = autoencoder.fit(
    train_dataset,
    epochs = 10,
    validation_data = val_dataset
)

Epoch 1/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - loss: 787.6461 - mae: 787.6461 - mse: 32630618.0000 - msle: 0.2840 - rmse: 5573.7593 - val_loss: 414.6924 - val_mae: 414.6924 - val_mse: 8918404.0000 - val_msle: 0.2465 - val_rmse: 2986.3696
Epoch 2/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 18s 8ms/step - loss: 335.5117 - mae: 335.5117 - mse: 6398744.5000 - msle: 0.2280 - rmse: 2529.2146 - val_loss: 427.6014 - val_mae: 427.6014 - val_mse: 9530455.0000 - val_msle: 0.2303 - val_rmse: 3087.1436
Epoch 3/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - loss: 303.8539 - mae: 303.8539 - mse: 5245147.0000 - msle: 0.2260 - rmse: 2289.9229 - val_loss: 399.2754 - val_mae: 399.2754 - val_mse: 7970514.5000 - val_msle: 0.2298 - val_rmse: 2823.2100
Epoch 4/10
2279/2279 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 291.5236 - mae: 291.5236 - mse: 4892400.5000 - msle: 0.2255 - rmse: 2211.6589 - val_loss: 445.3414 - val_mae: 445.3414 - val_mse: 9160513.0000 - val_msle: 0.2300 - val_rmse: 3026.6340
Epoch 5/10


In [40]:
instance = next(iter(test_read.map(prep_batch).batch(1).take(1)))[0].numpy()
instance

array([[ 4.33300000e+04, -1.51030791e+00,  7.80683602e-01,
         2.08574736e+00,  3.12313283e+00,  5.89556504e-01,
         1.30168094e+00, -1.30099101e+00, -2.34533311e+00,
         1.48722209e-01,  1.06300824e+00, -1.88035726e+00,
         3.10971711e-01,  9.54298638e-01, -1.10646280e+00,
        -7.86015840e-01, -5.55324333e-01,  3.35236557e-01,
         1.85637688e-01,  1.54380629e+00, -6.44864027e-01,
         2.22981820e+00, -4.60620924e-02, -3.41323962e-01,
        -3.70869344e-01, -2.01199298e-01,  4.23963954e-01,
         2.10998182e-01, -5.86163935e-02,  0.00000000e+00]])

In [41]:
autoencoder.predict(instance)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step


array([[ 4.1043676e+04, -4.3083081e-01,  3.4859994e-01,  9.0864396e-01,
         2.8234163e-01, -1.0169501e-01, -1.9621438e-01,  8.2337499e-02,
         5.1636048e-02, -5.3624310e-02, -1.8390632e-01,  2.1107332e-01,
         1.1533982e-01,  3.7093755e-02,  1.4556475e-01,  3.1027508e-01,
         9.8145328e-02, -3.0412519e-02, -1.2050614e-01, -4.4527762e-02,
        -2.6773306e-02, -1.2913945e-01, -2.2887911e-01, -4.8720278e-03,
         8.0836453e-02,  7.0976466e-02,  2.0507287e-02,  3.1646393e-02,
         2.8067801e-02,  1.0611104e+01]], dtype=float32)

---
Try #2 - Enhance

In [58]:
autoencoder_input = keras.Input(shape = (len(var_numeric),), name = "autoencoder_input")
normalized_input = normalizer(autoencoder_input)
encoder = keras.layers.Dense(32, activation='relu', name='enc_dense1')(normalized_input)
encoder = keras.layers.Dropout(0.2, name='enc_dropout1')(encoder)
encoder = keras.layers.Dense(16, activation='relu', name='enc_dense2')(encoder)
encoder = keras.layers.Dropout(0.2, name='enc_dropout2')(encoder)
encoder = keras.layers.Dense(8, activation='relu', name='enc_dense3')(encoder)
encoded = encoder
decoder = keras.layers.Dense(16, activation='relu', name='dec_dense1')(encoder)
decoder = keras.layers.Dropout(0.2, name='dec_dropout1')(decoder)
decoder = keras.layers.Dense(32, activation='relu', name='dec_dense2')(decoder)
decoder = keras.layers.Dropout(0.2, name='dec_dropout2')(decoder)
decoder = keras.layers.Dense(30, activation='linear', name='dec_dense3')(decoder)
reconstructed = denormalizer(decoder)

autoencoder = keras.Model(
    inputs = autoencoder_input, 
    outputs = {'reconstructed': reconstructed},
    name = 'autoencoder'
)

autoencoder.compile(
    optimizer = keras.optimizers.Adam(),
    loss = {'reconstructed': keras.losses.MeanAbsoluteError()},
    metrics = {
        'reconstructed': [
            keras.metrics.RootMeanSquaredError(name='rmse'),
            keras.metrics.MeanSquaredError(name='mse'),
            keras.metrics.MeanAbsoluteError(name='mae'),
            keras.metrics.MeanSquaredLogarithmicError(name='msle')
        ]
    }
)

# Data preparation for training
train_dataset = train_read.prefetch(10).map(prep_batch).batch(100)
val_dataset = validate_read.prefetch(10).map(prep_batch).batch(100)

history = autoencoder.fit(
    train_dataset,
    epochs = 10,
    validation_data = val_dataset,
)

ValueError: In the dict argument `metrics`, key 'reconstructed' does not correspond to any model output. Received:
metrics={'reconstructed': [<RootMeanSquaredError name=rmse>, <MeanSquaredError name=mse>, <MeanAbsoluteError name=mae>, <MeanSquaredLogarithmicError name=msle>]}

In [40]:
instance = next(iter(test_read.map(prep_batch).batch(1).take(1)))[0].numpy()
instance

array([[ 4.33300000e+04, -1.51030791e+00,  7.80683602e-01,
         2.08574736e+00,  3.12313283e+00,  5.89556504e-01,
         1.30168094e+00, -1.30099101e+00, -2.34533311e+00,
         1.48722209e-01,  1.06300824e+00, -1.88035726e+00,
         3.10971711e-01,  9.54298638e-01, -1.10646280e+00,
        -7.86015840e-01, -5.55324333e-01,  3.35236557e-01,
         1.85637688e-01,  1.54380629e+00, -6.44864027e-01,
         2.22981820e+00, -4.60620924e-02, -3.41323962e-01,
        -3.70869344e-01, -2.01199298e-01,  4.23963954e-01,
         2.10998182e-01, -5.86163935e-02,  0.00000000e+00]])

In [41]:
autoencoder.predict(instance)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step


array([[ 4.1043676e+04, -4.3083081e-01,  3.4859994e-01,  9.0864396e-01,
         2.8234163e-01, -1.0169501e-01, -1.9621438e-01,  8.2337499e-02,
         5.1636048e-02, -5.3624310e-02, -1.8390632e-01,  2.1107332e-01,
         1.1533982e-01,  3.7093755e-02,  1.4556475e-01,  3.1027508e-01,
         9.8145328e-02, -3.0412519e-02, -1.2050614e-01, -4.4527762e-02,
        -2.6773306e-02, -1.2913945e-01, -2.2887911e-01, -4.8720278e-03,
         8.0836453e-02,  7.0976466e-02,  2.0507287e-02,  3.1646393e-02,
         2.8067801e-02,  1.0611104e+01]], dtype=float32)